# Investigate a pure log fit to aerosol cloud interactions


\begin{equation}
F = \sum_{i} \beta_i \log \left( 1 + \frac{A_i}{n_i} \right)
\end{equation}


where 
- $A_i$ is the atmospheric input (concentrations or emissions of a specie),
- $\beta_i$ is a scale factor,
- $n_i$ is a shape factor that describes how linear or logarithmic the relationship is.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import json
import scipy.stats
import pooch
import os
import glob
from scipy.optimize import curve_fit

In [ ]:
files = glob.glob('../data/smith2021aerosol/*.csv')

ari = {}
aci = {}
models = []
years = {}
for file in files:
    model = os.path.split(file)[1][:-4]
    models.append(model)
    df = pd.read_csv(file, index_col='year')
    ari[model] = (df['aprp_ERFariSW'] + df['aprp_ERFariLW']).values.squeeze()
    aci[model] = (df['aprp_ERFaciSW'] + df['aprp_ERFaciLW']).values.squeeze()
    years[model] = df.index
#pd.read_csv('../data/smith2021aerosol/')

In [ ]:
int(np.floor(years['E3SM'][0]-1850))

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

In [ ]:
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
bc = emis_df.loc[(emis_df['Scenario']=='ssp245')&
                 (emis_df['Region']=='World')&
                 (emis_df['Variable']=='Emissions|BC'), '1750':'2100'].interpolate(axis=1).squeeze().values

oc = emis_df.loc[(emis_df['Scenario']=='ssp245')&
                 (emis_df['Region']=='World')&
                 (emis_df['Variable']=='Emissions|OC'), '1750':'2100'].interpolate(axis=1).squeeze().values

so2 = emis_df.loc[(emis_df['Scenario']=='ssp245')&
                 (emis_df['Region']=='World')&
                 (emis_df['Variable']=='Emissions|Sulfur'), '1750':'2100'].interpolate(axis=1).squeeze().values

In [ ]:
aci['UKESM1-0-LL']

In [ ]:
def aci_log(x, beta0, beta1, beta2, n0, n1, n2):
    aci = beta0*np.log(1 + x[0]/n0) + beta1*np.log(1 + x[1]/n1) + beta2*np.log(1 + x[2]/n2)
    aci_1850 = beta0*np.log(1 + so2[100]/n0) + beta1*np.log(1 + bc[100]/n1) + beta2*np.log(1 + oc[100]/n2)
    #aci_1750 = beta0*np.log(1 + so2[0]/n0) + beta1*np.log(1 + bc[0]/n1) + beta2*np.log(1 + oc[0]/n2)
    return (aci - aci_1850)

In [ ]:
param_fits = {}

for model in models:
    ist = int(np.floor(years[model][0]-1750))
    ien = int(np.ceil(years[model][-1]-1750))
    param_fits[model], cov = curve_fit(
        aci_log, 
        [so2[ist:ien], bc[ist:ien], oc[ist:ien]],
        aci[model],
        bounds = ((-np.inf, -np.inf, -np.inf, 0, 0, 0), (0, 0, 0, np.inf, np.inf, np.inf))
    )
#    p[0], p[1], p[2], p[3], p[4], p[5]

In [ ]:
fig, ax = pl.subplots(3, 4, figsize=(16, 12))
for imodel, model in enumerate(models):
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log([so2, bc, oc], *param_fits[model]))
    ax[imodel//4, imodel%4].plot(years[model], aci[model])

In [ ]:
df_params = pd.DataFrame(param_fits, index=['beta0', 'beta1', 'beta2', 'n0', 'n1', 'n2']).T

In [ ]:
def aci_log1750(x, beta0, beta1, beta2, n0, n1, n2):
    aci = beta0*np.log(1 + x[0]/n0) + beta1*np.log(1 + x[1]/n1) + beta2*np.log(1 + x[2]/n2)
    aci_1750 = beta0*np.log(1 + so2[0]/n0) + beta1*np.log(1 + bc[0]/n1) + beta2*np.log(1 + oc[0]/n2)
    return (aci - aci_1750)

In [ ]:
fig, ax = pl.subplots(3, 4, figsize=(16, 12))
for imodel, model in enumerate(models):
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log([so2, bc, oc], *param_fits[model]))
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log1750([so2, bc, oc], *param_fits[model]))

In [ ]:
fig, ax = pl.subplots(3, 4, figsize=(16, 12))
for imodel, model in enumerate(models):
    ax[imodel//4, imodel%4].plot(np.arange(1750.5, 2101), aci_log([so2, bc, oc], *param_fits[model]) - aci_log1750([so2, bc, oc], *param_fits[model]))

In [ ]:
df_params.to_csv('../data/smith2021aerosol/sumlog.csv')